In [27]:
import re, math, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler, QuantileTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from sklearn.isotonic import IsotonicRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import networkx as nx

from pyod.models.hbos import HBOS
from pyod.models.copod import COPOD

np.random.seed(42)

In [28]:
df = pd.read_excel('data/final_risk_data.xlsx')

# изменение типов данных
df['date'] = pd.to_datetime(df['date']).dt.normalize()
for i in ['debit_amount', 'credit_amount']:
    if i in df.columns:
        df[i] = pd.to_numeric(df[i], errors='coerce')
        
# создание колонки с суммой операции
df["amount"] = np.where(df["debit_amount"].notna(), df["debit_amount"], df["credit_amount"])
df["debit_amount"] = df["debit_amount"].fillna(0.0)
df["credit_amount"] = df["credit_amount"].fillna(0.0)

# созжание временных признаков
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
df['month'] = df['date'].dt.month
df["weekofyear"]   = df["date"].dt.isocalendar().week.astype(int)

# Суточные фичи
df['daily_total_debit'] = df.groupby(['debit_inn', 'date'])['debit_amount'].transform('sum')
df['daily_total_credit'] = df.groupby(['credit_inn', 'date'])['credit_amount'].transform('sum')
# посчет транзакций за сутки
df["daily_debit_transaction_count"]    = df.groupby(["debit_inn","date"])["debit_amount"].transform(lambda s: (s > 0).sum())
df["daily_credit_transaction_count"]    = df.groupby(["credit_inn","date"])["credit_amount"].transform(lambda s: (s > 0).sum())
# уникальные отправители и получатели в сутки
df["unique_recipients_per_day"] = df.groupby(["debit_inn","date"])["credit_inn"].transform("nunique")
df["unique_receivers_per_day"] = df.groupby(["credit_inn","date"])["debit_inn"].transform("nunique")

# Доля текущей операции в суточном объёме
df["daily_debit_percent"] = (df["debit_amount"] / df["daily_total_debit"].replace(0, np.nan)).fillna(0.0)
df["daily_credit_percent"] = (df["credit_amount"] / df["daily_total_credit"].replace(0, np.nan)).fillna(0.0)

# Интервалы между операциями (в днях)
df["days_since_last_txn_debit"]  = df.groupby("debit_inn")["date"].diff().dt.days.fillna(9999)
df["days_since_last_txn_credit"] = df.groupby("credit_inn")["date"].diff().dt.days.fillna(9999)

def add_rolling_side(df: pd.DataFrame,
                     side: str,
                     amt_col: str,
                     windows=(7, 14, 30, 90)) -> pd.DataFrame:
    """
    Создает роллинги по суммам/кол-ву транзакций за окна windows для заданной стороны.
    side: 'debit' или 'credit'
    amt_col: имя столбца с суммой для этой стороны (например, 'debit_amount' / 'credit_amount')
    """
    inn_col = f"{side}_inn"
    out_parts = []

    # Берем только нужные колонки (ускоряет groupby)
    need_cols = [inn_col, "date", amt_col]
    sub_all = df[need_cols].copy()
    sub_all[amt_col] = sub_all[amt_col].fillna(0.0)

    for inn, sub in sub_all.groupby(inn_col, sort=False):
        # суточная агрегация по аккаунту
        daily = (sub.groupby("date", as_index=True)[amt_col]
                   .sum()
                   .to_frame("amt_day")
                   .sort_index())

        # непрерывный календарный индекс (чтобы окна не "сжимались")
        idx = pd.date_range(daily.index.min(), daily.index.max(), freq="D")
        daily = daily.reindex(idx, fill_value=0.0)
        daily.index.name = "date"

        # индикатор "была ли операция в день" (для роллинга количества)
        daily["tx_day"] = (daily["amt_day"] > 0).astype(int)

        # роллинги
        for W in windows:
            daily[f"{side}_roll_sum_{W}d"]  = daily["amt_day"].rolling(W, min_periods=1).sum()
            daily[f"{side}_roll_cnt_{W}d"]  = daily["tx_day"].rolling(W, min_periods=1).sum()
            daily[f"{side}_roll_mean_{W}d"] = daily["amt_day"].rolling(W, min_periods=1).mean()
            daily[f"{side}_roll_std_{W}d"]  = daily["amt_day"].rolling(W, min_periods=1).std().fillna(0.0)
            daily[f"{side}_roll_p95_{W}d"]  = daily["amt_day"].rolling(W, min_periods=1).quantile(0.95)

        daily = daily.reset_index()
        daily[inn_col] = inn
        out_parts.append(daily[["date", inn_col] + [c for c in daily.columns if c.startswith(f"{side}_roll_")]])

    rolls = pd.concat(out_parts, ignore_index=True)
    return df.merge(rolls, on=["date", inn_col], how="left")



# Новые роллинги 7/14/30/90 для дебета и кредита
WINDOWS = (7, 14, 30, 90)
df = add_rolling_side(df, side="debit",  amt_col="debit_amount",  windows=WINDOWS)
df = add_rolling_side(df, side="credit", amt_col="credit_amount", windows=WINDOWS)

# ==== 1. Всплески сумм (amount_spike_ratio_7d) ====
df["debit_amount_spike_ratio_7d"]  = df["debit_roll_sum_7d"]  / (df["debit_roll_sum_30d"]/4 + 1e-6)
df["credit_amount_spike_ratio_7d"] = df["credit_roll_sum_7d"] / (df["credit_roll_sum_30d"]/4 + 1e-6)

# ==== 2. Всплески активности (tx_rate_spike_7d) ====
df["debit_tx_rate_spike_7d"]  = df["debit_roll_cnt_7d"]  / (df["debit_roll_cnt_30d"]/4 + 1e-6)
df["credit_tx_rate_spike_7d"] = df["credit_roll_cnt_7d"] / (df["credit_roll_cnt_30d"]/4 + 1e-6)

# ==== 3. Fan-out / Fan-in (уникальные контрагенты) ====
# Для дебета — fan-out (сколько разных получателей на операцию)
df["debit_fan_out_ratio"] = df["unique_recipients_per_day"] / (df["daily_debit_transaction_count"] + 1e-6)

# Для кредита — fan-in (сколько разных отправителей на операцию)
df["credit_fan_in_ratio"] = (
    df.groupby(["credit_inn", "date"])["debit_inn"].transform("nunique") /
    (df["daily_credit_transaction_count"] + 1e-6)
)

# ==== 4. Дисбаланс потоков (in_out_ratio_30d) ====
df["in_out_ratio_30d"] = (df["credit_roll_sum_30d"] + 1e-6) / (df["debit_roll_sum_30d"] + 1e-6)

# ==== 5. Волатильность объёмов (amount_volatility_30d) ====
df["debit_amount_volatility_30d"]  = df["debit_roll_std_30d"]  / (df["debit_roll_mean_30d"]  + 1e-6)
df["credit_amount_volatility_30d"] = df["credit_roll_std_30d"] / (df["credit_roll_mean_30d"] + 1e-6)


# Круглые суммы
df["round_10k"]  = ((df["amount"] % 10000)  == 0).astype(int)
df["round_100k"] = ((df["amount"] % 100000) == 0).astype(int)
df["round_large_amount"] = ((df["round_10k"]==1) | (df["round_100k"]==1)).astype(int)

df.head()



,date,debit_account,debit_name,debit_inn,credit_account,credit_name,credit_inn,debit_amount,credit_amount,purpose,...,debit_tx_rate_spike_7d,credit_tx_rate_spike_7d,debit_fan_out_ratio,credit_fan_in_ratio,in_out_ratio_30d,debit_amount_volatility_30d,credit_amount_volatility_30d,round_10k,round_100k,round_large_amount
0,2019-01-09,ded93f97f389bf2c,b736fcdbf591b1c2,d877722ca4e40f98,0c00203649ed677a,e19cc80da09f445f,8d9e0be733f77f1c,10.0,0.0,комиссия внутри сбербанка за пп/пт через дбо с...,...,3.999984,0.000000,1.0,1000000.0,3.333322e-13,0.000000,0.0,0,0,0
1,2019-01-09,ded93f97f389bf2c,b736fcdbf591b1c2,d877722ca4e40f98,7682fd5c32e028f5,796a88c244ffcc0a,6931e23e98703aa9,3000000.0,0.0,оплата по счету № 28 от date_9f241b636025 по д...,...,3.999984,0.000000,1.0,1000000.0,3.333322e-13,0.000000,0.0,1,1,1
2,2019-01-10,ded93f97f389bf2c,b736fcdbf591b1c2,d877722ca4e40f98,0c00203649ed677a,e19cc80da09f445f,8d9e0be733f77f1c,10.0,0.0,комиссия внутри сбербанка за пп/пт через дбо с...,...,3.999992,0.000000,1.0,1000000.0,3.328560e-13,1.410173,0.0,0,0,0
3,2019-01-10,ded93f97f389bf2c,b736fcdbf591b1c2,d877722ca4e40f98,e63cf19b76230a3d,21a73421ca00ae87,eed8b1f54ed4366c,4282.0,0.0,оплата по договору электроэнергия по дог.№1124...,...,3.999992,0.000000,1.0,1000000.0,3.328560e-13,1.410173,0.0,0,0,0
4,2019-01-10,d658b4e51c5a5df5,f7185cc66db08da5,ad57c94e5b8df8f6,ded93f97f389bf2c,3e368a63959d285e,d877722ca4e40f98,0.0,1000000.0,перевод денежных средств по договору займа № ч...,...,0.000000,3.999984,1000000.0,1.0,3.000000e+12,0.000000,0.0,1,1,1


In [29]:
# ============================================================
#  Анализ "Назначения платежа": TF-IDF (char 3–5) + стоп-слова риска
# ============================================================

import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# ------------------------------------------------------------
# 1) Регулярные паттерны высокого риска
# ------------------------------------------------------------
STOP_HIGH_PATTERNS = [
    r"\bзайм\w*\b", r"\bвозврат\W*(займ|долг)\w*\b", r"\bпогашен\w*\W*(займ|долг)\w*\b",
    r"\bдолг\w*\b", r"\bоплат\w*\W*(займ|долг)\w*\b",
    r"\bкрипт\w*\b", r"\bбиткоин\w*\b", r"\busdt\b", r"\bbtc\b", r"\bcoin\b",
    r"\bбирж\w*\b", r"\bобмен\w*\b", r"\bвалют\w*\b",
    r"\bналич\w*\b", r"\bобнал\w*\b", r"\bвыдач\w*\W*налич\w*\b",
    r"\bперевод\w*\W*(на|с)\W*карт\w*\b", r"\bперевод\w*\W*физ\w*\b",
    r"\bперевод\w*\W*родствен\w*\б", r"\bперевод\w*\W*средств\b",
    r"\bбез\W*договор\w*\b",
    r"\bпожертв\w*\b", r"\bблаготвор\w*\б", r"\bдарен\w*\b",
    r"\bагентск\w*\W*вознагражд\w*\б",
    # r"\bкомисси\w*\W*вознагражд\w*\b",
    r"\bвознагражд\w*\b",
    r"\bцесс\w*\б", r"\bпоручител\w*\b", r"\bзалог\w*\b",
    r"\bофшор\w*\б", r"\bиностран\w*\W*перевод\w*\b", r"\bswift\b",
    r"\bличн\w*\W*нужд\w*\б", r"\bпередач\w*\W*актив\w*\б",
    r"\bвклад\w*\б", r"\bдепозит\w*\б",
]
STOP_HIGH_RE = re.compile("|".join(STOP_HIGH_PATTERNS), flags=re.IGNORECASE)

# ------------------------------------------------------------
# 2) Очистка текста
# ------------------------------------------------------------
def clean_text(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"[^a-zа-я0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["purpose_clean"] = df["purpose"].apply(clean_text)

# ------------------------------------------------------------
# 3) Метка наличия стоп-слов риска
# ------------------------------------------------------------
df["purpose_stopword_high"] = df["purpose_clean"].str.contains(STOP_HIGH_RE, na=False)

# ------------------------------------------------------------
# 4) TF-IDF (char-level 3–5-граммы) + SVD
# ------------------------------------------------------------
texts = df["purpose_clean"].astype(str).tolist()

tfidf = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3, 5),
    min_df=1
)
X_tfidf = tfidf.fit_transform(texts)
print(f"TF-IDF матрица: {X_tfidf.shape[0]} × {X_tfidf.shape[1]}")

# SVD для сжатия (например, до 50 компонент)
svd_k_target = 50
max_svd = max(1, min(X_tfidf.shape[0] - 1, X_tfidf.shape[1] - 1))
n_svd = min(svd_k_target, max_svd)

svd = TruncatedSVD(n_components=n_svd, random_state=42)
X_svd = svd.fit_transform(X_tfidf)

for i in range(X_svd.shape[1]):
    df[f"purpose_svd_{i+1}"] = X_svd[:, i]

expl_var = svd.explained_variance_ratio_.sum()
print(f"SVD explained variance (k={X_svd.shape[1]}): {expl_var:.3f}")

# ------------------------------------------------------------
# 5) Результат
# ------------------------------------------------------------
svd_cols = [c for c in df.columns if c.startswith("purpose_svd_")]
cols_show = ["purpose", "purpose_stopword_high"] + svd_cols

print(f"\n Добавлены признаки: {len(svd_cols)} SVD-компонент + метка стоп-слов")
print(df[cols_show].head())


TF-IDF матрица: 2688 × 45426
SVD explained variance (k=50): 0.545

 Добавлены признаки: 50 SVD-компонент + метка стоп-слов
                                             purpose  purpose_stopword_high  \
0  комиссия внутри сбербанка за пп/пт через дбо с...                  False   
1  оплата по счету № 28 от date_9f241b636025 по д...                  False   
2  комиссия внутри сбербанка за пп/пт через дбо с...                  False   
3  оплата по договору электроэнергия по дог.№1124...                  False   
4  перевод денежных средств по договору займа № ч...                   True   

   purpose_svd_1  purpose_svd_2  purpose_svd_3  purpose_svd_4  purpose_svd_5  \
0       0.708416      -0.202923      -0.110215      -0.029243       0.142140   
1       0.122677       0.236405      -0.200400      -0.017372      -0.097021   
2       0.602413      -0.212909      -0.103785      -0.020422       0.124845   
3       0.108051       0.160672      -0.135616      -0.000770      -0.035201   
4 

In [30]:
# =========================
# GCN Autoencoder (GAE) для AML на транзакциях по ИНН
# С фильтрацией по участию ИНН в графе (MIN_DEG)
# =========================

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from tqdm import tqdm

# -------------------------
# ПАРАМЕТРЫ
# -------------------------
MIN_DEG = 1  # количество контрагентов
EPOCHS = 300 # сколько раз нейросеть пройдет весь граф при обучении 
LR = 1e-3 # показывает насколько сильно обновляются веса в нейросети на каждом шаге
WDECAY = 1e-4 # маленький штраф за слишком большие веса
HIDDEN = 64 #
LATENT = 32 #
DROPOUT = 0.3 #
NEG_POS_RATIO = 1.0  # сколько негативных пар на каждое позитивное ребро

device = torch.device('cpu')

# -------------------------
# 1) Построение графа ИНН–ИНН и фич узлов
# -------------------------
# Ожидается DataFrame df со столбцами, как ты присылал (debit_inn/credit_inn и числовые признаки).

inn_series = pd.concat([df['debit_inn'], df['credit_inn']], ignore_index=True)
unique_inn = pd.Index(inn_series.dropna().unique())
inn2id = {inn: i for i, inn in enumerate(unique_inn)}
num_nodes = len(unique_inn)

edges_src_s = df['debit_inn'].map(inn2id)
edges_dst_s = df['credit_inn'].map(inn2id)
mask_valid = (edges_src_s.notna() & edges_dst_s.notna())
edges_src = edges_src_s[mask_valid].astype(int).values
edges_dst = edges_dst_s[mask_valid].astype(int).values

# Неориентированный граф для энкодера (добавим обратные рёбра):
edge_index = torch.tensor([edges_src, edges_dst], dtype=torch.long)
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

# Выберем числовые признаки для профиля ИНН
exclude_cols = [
    'date','debit_account','debit_name','debit_inn',
    'credit_account','credit_name','credit_inn','purpose'
] + [f"purpose_svd_{i}" for i in range(1,51)]

numeric_cols = [c for c in df.columns
                if c not in exclude_cols and pd.api.types.is_numeric_dtype(df[c])]

df_num = df[numeric_cols].copy().fillna(0)

# Агрегации по ИНН (как дебитор и как кредитор), затем усредняем
agg_debit = df.groupby('debit_inn')[numeric_cols].mean().reindex(unique_inn, fill_value=0)
agg_credit = df.groupby('credit_inn')[numeric_cols].mean().reindex(unique_inn, fill_value=0)
node_features = (agg_debit.values + agg_credit.values) / 2.0

# Масштабирование
scaler = StandardScaler()
node_features = scaler.fit_transform(node_features)

x = torch.tensor(node_features, dtype=torch.float, device=device)
ei = edge_index.to(device)

data_pg = Data(x=x, edge_index=ei.cpu())  # для печати размеров
print(data_pg)

# -------------------------
# 2) Модель и torch-only negative sampling
# -------------------------
class GCNEncoder(nn.Module):
    def __init__(self, in_ch, hidden=64, out_ch=32, dropout=0.2):
        super().__init__()
        self.conv1 = GCNConv(in_ch, hidden)
        self.conv2 = GCNConv(hidden, out_ch)
        self.dropout = dropout

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)
        z = self.conv2(h, edge_index)
        return z

def torch_negative_sampling(edge_index: torch.Tensor, num_nodes: int, num_neg_samples: int):
    """Чисто Torch негативный сэмплинг: выдаёт пары (u,v), которых нет в edge_index и u!=v."""
    device = edge_index.device
    u = edge_index[0].to(torch.long)
    v = edge_index[1].to(torch.long)
    enc_exist = (u * num_nodes + v).unique()
    got = 0
    chunks = []
    batch = max(1024, min(num_neg_samples, 65536))
    while got < num_neg_samples:
        uu = torch.randint(0, num_nodes, (batch,), device=device)
        vv = torch.randint(0, num_nodes, (batch,), device=device)
        enc = uu * num_nodes + vv
        mask = (~torch.isin(enc, enc_exist)) & (uu != vv)
        if mask.any():
            pair = torch.stack([uu[mask], vv[mask]], dim=0)
            chunks.append(pair[:, : (num_neg_samples - got)])
            got += chunks[-1].size(1)
        if batch < num_neg_samples:
            batch = min(num_neg_samples, batch * 2)
    return torch.cat(chunks, dim=1)

def recon_loss(z, edge_index, num_nodes, neg_ratio=1.0):
    pos_i, pos_j = edge_index
    num_neg = int(pos_i.numel() * neg_ratio)
    neg_edge_index = torch_negative_sampling(edge_index, num_nodes=num_nodes, num_neg_samples=num_neg)

    pos_logits = (z[pos_i] * z[pos_j]).sum(dim=-1)
    neg_logits = (z[neg_edge_index[0]] * z[neg_edge_index[1]]).sum(dim=-1)

    pos_loss = F.binary_cross_entropy_with_logits(pos_logits, torch.ones_like(pos_logits))
    neg_loss = F.binary_cross_entropy_with_logits(neg_logits, torch.zeros_like(neg_logits))
    return pos_loss + neg_loss

model = GCNEncoder(in_ch=x.size(1), hidden=HIDDEN, out_ch=LATENT, dropout=DROPOUT).to(device)
opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WDECAY)

# -------------------------
# 3) Обучение
# -------------------------
for ep in tqdm(range(1, EPOCHS + 1), desc="Training", unit="epoch"):
    model.train()
    opt.zero_grad()
    z = model(x, ei)
    loss = recon_loss(z, ei, num_nodes=num_nodes, neg_ratio=NEG_POS_RATIO)
    loss.backward()
    opt.step()
    if ep % 50 == 0 or ep == 1:
        tqdm.write(f"Epoch {ep:03d} | Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    z = model(x, ei)

# -------------------------
# 4) Скоринг аномалий (с фильтром по участию ИНН)
# -------------------------
sigmoid = torch.sigmoid

# Исходные направленные рёбра (без обратных), для скоринга рёбер:
orig_src = torch.tensor(edges_src, dtype=torch.long, device=device)
orig_dst = torch.tensor(edges_dst, dtype=torch.long, device=device)

with torch.no_grad():
    edge_logits = (z[orig_src] * z[orig_dst]).sum(dim=-1)
    edge_p = sigmoid(edge_logits)
edge_score = (1.0 - edge_p).cpu().numpy()  # выше — подозрительнее

# Участие каждого ИНН (сколько раз встречается как источник/получатель в исходных рёбрах)
participation = np.bincount(
    np.concatenate([edges_src, edges_dst]),
    minlength=num_nodes
)

# Рёбра учитываем только если оба конца имеют участие >= MIN_DEG
keep_edge_mask = (participation[edges_src] >= MIN_DEG) & (participation[edges_dst] >= MIN_DEG)

edges_src_f = edges_src[keep_edge_mask]
edges_dst_f = edges_dst[keep_edge_mask]
edge_score_f = edge_score[keep_edge_mask]

# Top-20 подозрительных связей (агрегация по (debit_inn, credit_inn))
edge_keys = pd.MultiIndex.from_arrays(
    [df.loc[mask_valid, 'debit_inn'].values[keep_edge_mask],
     df.loc[mask_valid, 'credit_inn'].values[keep_edge_mask]],
    names=['debit_inn','credit_inn']
)
edge_df = pd.DataFrame({'edge_score': edge_score_f}, index=edge_keys).groupby(level=[0,1]).mean()
edge_top = edge_df.sort_values('edge_score', ascending=False).head(20)
print(f"\nTop-20 подозрительных связей (ИНН→ИНН), порог участия узлов ≥ {MIN_DEG}:")
display(edge_top)

# --- Узловые скора ---
num_nodes = int(num_nodes)
node_accum = np.zeros(num_nodes, dtype=np.float64)
node_cnt   = np.zeros(num_nodes, dtype=np.int64)

# учитываем только отфильтрованные рёбра
for s, d, sc in zip(edges_src_f, edges_dst_f, edge_score_f):
    node_accum[s] += sc; node_cnt[s] += 1
    node_accum[d] += sc; node_cnt[d] += 1

node_score_edges = np.divide(node_accum, np.maximum(node_cnt, 1))

# изолированность по косинусу (на неориентированном графе), потом отфильтруем узлы ниже порога
with torch.no_grad():
    zn = F.normalize(z, dim=1)

cos_accum = np.zeros(num_nodes, dtype=np.float64)
cos_cnt   = np.zeros(num_nodes, dtype=np.int64)
ei_cpu = ei.detach().cpu().numpy()

for u, v in zip(ei_cpu[0], ei_cpu[1]):
    cos = float((zn[u] * zn[v]).sum().cpu())
    cos_accum[u] += cos; cos_cnt[u] += 1

node_isolation = 1.0 - np.divide(cos_accum, np.maximum(cos_cnt, 1))

# Узлам с участием < MIN_DEG не присваиваем скор (NaN)
node_score_edges[participation < MIN_DEG] = np.nan
node_isolation[participation < MIN_DEG]   = np.nan

node_tbl = pd.DataFrame({
    'inn': unique_inn.values,
    'participation': participation,
    'edge_based_score': node_score_edges,
    'isolation_score': node_isolation
}).set_index('inn')
node_tbl['node_score'] = 0.5*node_tbl['edge_based_score'] + 0.5*node_tbl['isolation_score']

node_top = node_tbl.dropna(subset=['node_score']) \
                   .sort_values('node_score', ascending=False).head(20)

print(f"\nTop-20 подозрительных ИНН (узлы с участием ≥ {MIN_DEG}):")
display(node_top)

# -------------------------
# 5) Примечания
# -------------------------
# - Меняй MIN_DEG для контроля "мусорных" узлов с единичными транзакциями.
# - Для очень больших графов снизь EPOCHS или используй GraphSAGE с neighbor sampling.
# - Если важно направление и/или фичи рёбер, сделай декодер-MLP на [z_u, z_v, edge_features].


Data(x=[111, 71], edge_index=[2, 5348])


Training:   5%|███▎                                                               | 15/300 [00:00<00:01, 144.51epoch/s]

Epoch 001 | Loss: 1.1256


Training:  21%|██████████████                                                     | 63/300 [00:00<00:01, 156.48epoch/s]

Epoch 050 | Loss: 0.7406


...

Training: 100%|██████████████████████████████████████████████████████████████████| 300/300 [00:01<00:00, 163.40epoch/s]

Epoch 300 | Loss: 0.7762

Top-20 подозрительных связей (ИНН→ИНН), порог участия узлов ≥ 1:


edge_score
debit_inn        credit_inn                  
032f0b4d2498f263 d877722ca4e40f98    0.027253
d877722ca4e40f98 3a9c5401f637dd8d    0.027253
                 813f62b60a645969    0.027253
                 808f1347a5fad97c    0.027253
                 7b6d655993f0d2ed    0.027253
                 7ad54077510c09e4    0.027253
                 77af13772af29f14    0.027253
                 777e4d144e99f74a    0.027253
                 767c3f5f72b4494f    0.027253
                 74b54be399407640    0.027253
                 71aea28b46e14023    0.027253
                 6ef5667d7851dcbc    0.027253
                 6e8af9f6fa6a662b    0.027253
                 6d806e49c5774c77    0.027253
                 699e6f31660016bf    0.027253
                 683c3112f4fb5700    0.027253
                 659207c64fee1c66    0.027253
                 5a0eeb2c5b3da864    0.027253
                 582da928bb55efa7    0.027253
                 532f3d84ba4caf61    0.027253


Top-20 подозрительных ИНН (узлы с участием ≥ 1):


,participation,edge_based_score,isolation_score,node_score
inn,,,,
333706e1b48a76ee,1,0.000000,0.932389,0.466195
516b9fe460a8ab7c,1,0.000000,0.917026,0.458513
524ef6bc49f42742,13,0.000647,0.725176,0.362911
9e2cf0cc6f3d68e4,1,0.027253,0.407757,0.217505
ed18d013d231f2d0,12,0.027253,0.407757,0.217505
20b5688cbd9506bf,6,0.027253,0.407757,0.217505
0a10966a0ec7c164,2,0.027253,0.407757,0.217505
8a33d5e5ee28c48d,1,0.027253,0.407757,0.217505
225acb9ed4f34af6,14,0.027253,0.407757,0.217505
